In [11]:
from pyngrok import ngrok
ngrok.kill()

In [7]:
! ngrok authtoken YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
ngrok.connect(5000)

<NgrokTunnel: "http://508780ad039d.ngrok.io" -> "http://localhost:5000">

In [4]:
!apt-get install portaudio19-dev python3-pyaudio

In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 9.8MB 4.3MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 3.2MB 37.1MB/s 
     |████████████████████████████████| 184kB 41.7MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 44.7MB 75kB/s 
     |████████████████████████████████| 10.3MB 15.5MB/s 
     |████████████████████████████████| 14.5MB 293kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 3.0MB 26.8MB/s 
     |████████████████████████████████| 9.1MB 33.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 727kB 38.6MB/s 
     |████████████████████████████████| 901kB 32.2MB/s 
     |████████████████████████████████| 22.3MB 50.9MB/s 
     |████████████████████████████████| 27.4MB 163kB/s 
     |████████████████████████████████| 1.2MB 27.5MB/s 
     |████████████████████████████████| 122kB 44.8MB/s 

In [9]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
from flask import Flask
import datetime
from flask import request # used to parse payload
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
from flask import render_template
from flask import abort
from flask_cors import CORS
import os

# define a variable to hold you app
app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return render_template('index.html')

@app.route('/time', methods=['GET'])
def get_time():
    return str(datetime.datetime.now())

@app.route('/api/summarize', methods=['GET'])
def GetUrl():
    """
    Called as /api/summarize?youtube_url='url'
    """
    # if user sends payload to variable name, get it. Else empty string
    video_url = request.args.get('youtube_url', '') 
    # if(len(video_url) == 0) or (not '=' in video_url):
    #   print("f")
    #   abort(404)
    
    response = GetTranscript(video_url)
    return response

def SumySummarize(text):

    from sumy.parsers.html import HtmlParser
    from sumy.parsers.plaintext import PlaintextParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.lsa import LsaSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 10
    import nltk;  
    nltk.download('punkt')

    # url = "https://en.wikipedia.org/wiki/Automatic_summarization"
    # parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    s = ""
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
      s += (str)(sentence)
      s += "\n"
    print('The summary is: ', s)
    return s

def GetTextFromAudio():
    import speech_recognition as sr
    from pydub import AudioSegment
    
    f = ""

    # convert mp3 file to wav           
    for file in os.listdir(os.getcwd()):
      if file.endswith(".mp3"):
          f = os.path.join(os.getcwd(), file)
          print(os.path.join(os.getcwd(), file))      
                                          
    if(len(f) == 0):
      return f
    print('The file is: ', f)
    sound = AudioSegment.from_mp3(f)

    os.remove(f)
    
    sound.export("transcript.wav", format="wav")
    
    # use the audio file as the audio source                                        
    AUDIO_FILE = "transcript.wav"

    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
      audio = r.record(source)  # read the entire audio file    
      print(audio)              
      return (r.recognize_google(audio))

def GetAudio(video_url):
    from youtube_dl import YoutubeDL
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3'
        }],
    }
    with YoutubeDL(ydl_opts) as ydl:
       ydl.download([video_url])
      

# video id are the last characters in the link of youtube video
def GetTranscript(video_url): 
    text = ""
    try:
      video_id = video_url.split('=')[1]
      transcript = YouTubeTranscriptApi.get_transcript(video_id)
      formatter = TextFormatter()
      text = formatter.format_transcript(transcript)
      return SumySummarize(text)
    except:
      GetAudio(video_url)
      text = GetTextFromAudio()
      print('The text is: ', text)
      return SumySummarize(text)

# server the app when this file is run
if __name__ == '__main__': 
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/May/2021 04:03:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2021 04:03:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/May/2021 04:03:46] "GET / HTTP/1.1" 200 -


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


127.0.0.1 - - [15/May/2021 04:03:51] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=sXdfRYyzbmU HTTP/1.1" 200 -


The summary is:  but people are not objects on an assembly line that's a ridiculous notion [Music] what would you do if you had a free reign over education generally you want education to be like as close to a video game as possible like a good video game like you do not need to tell your kid to play video games they will play video games on autopilot all day so if you can make it interactive and engaging then then you can make education far more compelling and far easier to do so you really want to um disconnect the whole grade level three thing from the subjects allow people to progress at the fastest pace that they can or are interested in in each subject it seems like a really obvious thing um i mean i think like most teaching today is is a lot like vordeville where um and it's and as a result just not that compelling it's like somebody's standing up there and lecturing to you and they've done the same lecture several years in a row they're not necessarily all that engaged or or in

127.0.0.1 - - [15/May/2021 04:05:17] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=sXdfRYyzbmU HTTP/1.1" 200 -


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The summary is:  but people are not objects on an assembly line that's a ridiculous notion [Music] what would you do if you had a free reign over education generally you want education to be like as close to a video game as possible like a good video game like you do not need to tell your kid to play video games they will play video games on autopilot all day so if you can make it interactive and engaging then then you can make education far more compelling and far easier to do so you really want to um disconnect the whole grade level three thing from the subjects allow people to progress at the fastest pace that they can or are interested in in each subject it seems like a really obvious thing um i mean i think like most teaching today is is a lot like vordeville where um and it's and as a result just not that compelling it's like somebody's standing up there and lecturing to

127.0.0.1 - - [15/May/2021 04:05:45] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=rfpuhFXWwe8 HTTP/1.1" 200 -


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The summary is:  fly people to overton back and um i mean i really came here because i just wanted to see bum duck to be totally fine like thank goodness um like my entire adrenaline it was just just dumped you know thank god you know and i'm not very religious but i prayed for this one um so it's once again thanks everyone uh spacex nasa f a ever everyone f4 suit it you know that that played a role in in this and not its key suppliers that that did incredible work thanks thanks again and you know i think this is something that the whole world can take uh some some pleasure in and and can really look at this as an achievement of humanity

[youtube] irPJTOBOo3c: Downloading webpage
[download] Destination: A Day In The Life Of Elon Musk-irPJTOBOo3c.webm
[download] 100% of 9.06MiB in 00:00
[ffmpeg] Destination: A Day In The Life Of Elon Musk-irPJTOBOo3c.mp3
Deleting original file

[2021-05-15 04:06:24,637] ERROR in app: Exception on /api/summarize [GET]
Traceback (most recent call last):
  File "<ipython-input-9-d3640f8f1dac>", line 141, in GetTranscript
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
  File "/usr/local/lib/python3.7/dist-packages/youtube_transcript_api/_api.py", line 128, in get_transcript
    return cls.list_transcripts(video_id, proxies, cookies).find_transcript(languages).fetch()
  File "/usr/local/lib/python3.7/dist-packages/youtube_transcript_api/_api.py", line 70, in list_transcripts
    return TranscriptListFetcher(http_client).fetch(video_id)
  File "/usr/local/lib/python3.7/dist-packages/youtube_transcript_api/_transcripts.py", line 36, in fetch
    self._extract_captions_json(self._fetch_video_html(video_id), video_id)
  File "/usr/local/lib/python3.7/dist-packages/youtube_transcript_api/_transcripts.py", line 48, in _extract_captions_json
    raise TranscriptsDisabled(video_id)
youtube_transcript_api._errors.Transcript